In [9]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get POSTGRES_ADDRESS from .env file
POSTGRES_ADDRESS = os.getenv("POSTGRES_ADDRESS")

# Create engine with the address
engine = create_engine(POSTGRES_ADDRESS)

df = pd.read_sql_query("SELECT * FROM benchmark_runs", engine)
# Set pandas display options to make tables wider
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)        # Set width to 1000 characters
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple lines
# Drop the 'args' column if it exists in the DataFrame
# if 'args' in df.columns:
df = df.drop(columns=['config_details_json'])
print(df.columns)


Index(['id', 'timestamp', 'input_file', 'output_dir', 'notes', 'duplicate_count', 'record_count', 'implementation', 'num_nodes', 'threshold', 'ngram_size', 'min_ngram_size', 'num_perm', 'execution_time', 'limit_files', 'total_size_gb', 'nd_time_sec', 'nd_output_count', 'config_file_path', 'cl_train_time_sec', 'cl_inference_time_sec', 'cl_stage2_time_sec'], dtype='object')


In [10]:
df

,id,timestamp,input_file,output_dir,notes,duplicate_count,record_count,implementation,num_nodes,threshold,ngram_size,min_ngram_size,num_perm,execution_time,limit_files,total_size_gb,nd_time_sec,nd_output_count,config_file_path,cl_train_time_sec,cl_inference_time_sec,cl_stage2_time_sec
0,1,2025-04-11 10:51:27.842752,/dev/shm/c4_files/c4-train.*.json.gz,/dev/shm/c4_outputs,Workflow: cl_nd | CL Cfg: base.yml,3241,353077,cl_nd,10,0.7,5,5,256,302.567895,1,0.297670,0.000000,NaN,database_project/src/configs/base.yml,156.555996,132.689848,233.293692
1,2,2025-04-11 11:15:25.118859,/dev/shm/c4_files/c4-train.*.json.gz,/mnt/gcs_bucket/ray_experiment_outputs_2025041...,Experiment: datasize | Workflow: nd_cl | Limit...,76548,3486625,nd_cl,10,0.7,5,5,256,570.027176,10,2.972425,69.257010,3486625.0,/home/ohadr/database_project_c/database_projec...,1439.614951,150.740207,336.512912
2,3,2025-04-11 11:24:34.617626,/dev/shm/c4_files/c4-train.*.json.gz,/mnt/gcs_bucket/ray_experiment_outputs_2025041...,Experiment: datasize | Workflow: cl_nd | Limit...,73958,3489215,cl_nd,10,0.7,5,5,256,530.176133,10,2.972425,0.000000,NaN,/home/ohadr/database_project_c/database_projec...,1414.367524,173.493992,344.237418
3,4,2025-04-11 11:35:28.946751,/dev/shm/c4_files/c4-train.*.json.gz,/mnt/gcs_bucket/ray_experiment_outputs_2025041...,Experiment: datasize | Workflow: nd_cl | Limit...,180792,6945553,nd_cl,10,0.7,5,5,256,636.150529,20,5.942862,74.634385,6945553.0,/home/ohadr/database_project_c/database_projec...,1437.450196,200.961973,389.771242
4,5,2025-04-11 11:45:00.044283,/dev/shm/c4_files/c4-train.*.json.gz,/mnt/gcs_bucket/ray_experiment_outputs_2025041...,Experiment: datasize | Workflow: cl_nd | Limit...,175302,6951043,cl_nd,10,0.7,5,5,256,551.382343,20,5.942862,0.000000,NaN,/home/ohadr/database_project_c/database_projec...,1439.514931,212.965525,358.743246
5,6,2025-04-11 11:58:22.989802,/dev/shm/c4_files/c4-train.*.json.gz,/mnt/gcs_bucket/ray_experiment_outputs_2025041...,Experiment: datasize | Workflow: nd_cl | Limit...,421609,13831082,nd_cl,10,0.7,5,5,256,784.469146,40,11.883736,95.636764,13831082.0,/home/ohadr/database_project_c/database_projec...,1418.278177,292.422638,503.903034
6,7,2025-04-11 12:10:59.699646,/dev/shm/c4_files/c4-train.*.json.gz,/mnt/gcs_bucket/ray_experiment_outputs_2025041...,Experiment: datasize | Workflow: cl_nd | Limit...,409595,13843096,cl_nd,10,0.7,5,5,256,735.537131,40,11.883736,0.000000,NaN,/home/ohadr/database_project_c/database_projec...,1452.173147,296.034712,526.081583
7,8,2025-04-11 12:24:45.873177,/dev/shm/c4_files/c4-train.*.json.gz,/mnt/gcs_bucket/ray_experiment_outputs_2025041...,Experiment: threshold | Workflow: nd_cl | Limi...,547957,13704734,nd_cl,10,0.6,5,5,256,805.386811,40,11.883736,108.114633,13704734.0,/home/ohadr/database_project_c/database_projec...,1435.105881,291.325886,514.827636
8,9,2025-04-11 12:37:10.812378,/dev/shm/c4_files/c4-train.*.json.gz,/mnt/gcs_bucket/ray_experiment_outputs_2025041...,Experiment: threshold | Workflow: cl_nd | Limi...,527329,13725362,cl_nd,10,0.6,5,5,256,724.325721,40,11.883736,0.000000,NaN,/home/ohadr/database_project_c/database_projec...,1446.516445,306.474160,517.582288
9,10,2025-04-11 12:50:16.390571,/dev/shm/c4_files/c4-train.*.json.gz,/mnt/gcs_bucket/ray_experiment_outputs_2025041...,Experiment: threshold | Workflow: nd_cl | Limi...,421609,13831082,nd_cl,10,0.7,5,5,256,765.237342,40,11.883736,106.790467,13831082.0,/home/ohadr/database_project_c/database_projec...,1426.093020,315.023868,479.225382


In [11]:

df.iloc[1:].to_csv("benchmark_runs.csv", index=False)

In [ ]:
df.execution_time.sum()/3600

In [ ]:
df.duplicate_count+df.record_count

In [ ]:
2.7*60

In [ ]:
752.379005/

In [ ]:
421_609

In [ ]:
13831082

In [ ]:
13_831_082

In [ ]:
200/

In [ ]:
1_749_756

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("/home/ohadr/database_project_c/benchmark_results.db")

df = pd.read_sql_query("SELECT * FROM benchmark_runs", conn)
df

In [ ]:
print(df[["duplicate_count","record_count","execution_time", "total_size_gb"]].to_markdown())